In [ ]:
%%bash
git add nlp100_80.ipynb

In [ ]:
%%bash
git commit -a -m "snapshot"

****0 コーパスの整形 ****


文を単語列に変換する最も単純な方法は，空白文字で単語に区切ることである． ただ，この方法では文末のピリオドや括弧などの記号が単語に含まれてしまう． そこで，コーパスの各行のテキストを空白文字でトークンのリストに分割した後，各トークンに以下の処理を施し，単語から記号を除去せよ．

- トークンの先頭と末尾に出現する次の文字を削除: .,!?;:()[]'"
- 空文字列となったトークンは削除

以上の処理を適用した後，トークンをスペースで連結してファイルに保存せよ

In [1]:
import bz2

In [2]:
import codecs

In [3]:
fname_input = "nlp100data/enwiki-20150112-400-r100-10576.txt.bz2"

In [4]:
fname_output = "nlp100data/corpus80.txt"

- bz2.open の引数"rt"の"t"はtextを表す。省略するとbyte型になってしまう。
- chunk.strip()は引数を指定しないとwhitespaceを除去

In [ ]:
with bz2.open(fname_input  , 'rt') as rf , codecs.open(fname_output , "w" , "utf-8") as wf:
    for i , l in enumerate(rf):
        if i > -1:
            tokens = []
            for chunk in l.split(' '):
                token = chunk.strip().strip('.,!?;:()[]\'"')
                if len(token) > 0:
                    tokens.append(token)
                    
            processed = " ".join(tokens) + "\n"
            wf.write(processed)

                


In [ ]:
 " ".join(tokens) + "\n"

#### 複合語からなる国名への対処
英語では，複数の語の連接が意味を成すことがある．例えば，アメリカ合衆国は"United States"，イギリスは"United Kingdom"と表現されるが，"United"や"States"，"Kingdom"という単語だけでは，指し示している概念・実体が曖昧である．そこで，コーパス中に含まれる複合語を認識し，複合語を1語として扱うことで，複合語の意味を推定したい．しかしながら，複合語を正確に認定するのは大変むずかしいので，ここでは複合語からなる国名を認定したい．

インターネット上から国名リストを各自で入手し，80のコーパス中に出現する複合語の国名に関して，スペースをアンダーバーに置換せよ．例えば，"United States"は"United_States"，"Isle of Man"は"Isle_of_Man"になるはずである．

https://gist.github.com/kalinchernev/486393efcca01623b18d

In [5]:
import re

In [6]:
fname_output81 = "nlp100data/output81.txt"

In [ ]:
%%bash
cd nlp100data

wget https://gist.githubusercontent.com/kalinchernev/486393efcca01623b18d/raw/daa24c9fea66afb7d68f8d69f0c4b8eeb9406e83/countries

In [ ]:
with codecs.open("nlp100data/countries"
                 , "r" , "utf-8") as rf:
    country_names = [l.strip('\n') for l in rf]

    

In [ ]:
country_names

In [ ]:
len(country_names)

In [ ]:
%%bash
git commit -a -m "doing 81"

In [ ]:
compound_countries = dict()
for c in country_names:
    c_split = c.split(" ")
    n = len(c_split)
    if n > 1:
        compound_countries[c] = "_".join(c_split)
        

In [ ]:
with codecs.open(fname_output , "r" , "utf-8") as rf , codecs.open(fname_output81 , "w" , "utf-8") as wf:
    for i, l in enumerate(rf):
        for k ,v in compound_countries.items():
            l = l.replace(k , v)
        #print(l)
        wf.write(l)
        #if i > 10:
         #   break

In [ ]:
%%bash
git commit -a -m "finished 81"

##### sandbox

In [ ]:
with codecs.open(fname_output81 , "r" , "utf-8") as rf:
    for i, l in enumerate(rf):
        print(l)
        if i > 3:
            break

In [ ]:
for k  , v in compound_countries.items():
    print(k , v)

#### 文脈の抽出
81で作成したコーパス中に出現するすべての単語$t$に関して，単語$t$と文脈語$c$のペアをタブ区切り形式ですべて書き出せ．ただし，文脈語の定義は次の通りとする．

- ある単語$t$の前後$d$単語を文脈語$c$として抽出する（ただし，文脈語に単語$t$そのものは含まない）
- 単語$t$を選ぶ度に，文脈幅$d$は{1,2,3,4,5}の範囲でランダムに決める．





In [7]:
import pickle

In [8]:
import random

In [9]:
fname_list82 = "nlp100data/list82.pkl"

In [ ]:
random.randint(1,5)

In [ ]:
list82 = list()
with codecs.open(fname_output81 , "r" , "utf-8") as rf:
    for i, l in enumerate(rf):
        v = l.strip().split(' ')
        
        
        for j , w in enumerate(v):
            d = random.randint(1 , 5)
            vcontext = [v[k] for k in set(range(j-d , j + d + 1)) & set(range(len(v))) - set([j]) ]
            if len(w) * len(vcontext) >0:
                list82.append(w + "\t" + " ".join(vcontext))
        if i > 99999999999:
            break

In [ ]:
with open(fname_list82 , "wb") as wf:
    pickle.dump(list82 , wf)

In [ ]:
len(tmp)


In [ ]:
len(list82)

In [ ]:
[print(l) for i , l in enumerate(list82) if i < 10] 

##### sandbox

In [ ]:
len(vcontext)

In [ ]:
w.strip?

In [ ]:
%%bash
git commit -a -m "done 82"

In [ ]:
with codecs.open(fname_output81 , "r" , "utf-8") as rf:
    for i, l in enumerate(rf):
        print(l)
        if i > 3:
            break

#### 単語／文脈の頻度の計測
82の出力を利用し，以下の出現分布，および定数を求めよ．

- $f(t,c)$: 単語$t$と文脈語$c$の共起回数
- $f(t,∗)$: 単語$t$の出現回数
- $f(∗,c)$: 文脈語$c$の出現回数
- $N$:単語と文脈語のペアの総出現回数

In [10]:
fname_tc_counter83 = "nlp100data/tc_counter83.pkl"
fname_t_counter83 = "nlp100data/t_counter83.pkl"
fname_c_counter83 = "nlp100data/c_counter83.pkl"

In [11]:
import pickle

In [12]:
from collections import Counter

In [ ]:
with open(fname_list82 , "rb") as rf:
    list82 = pickle.load(rf)

In [ ]:
tc_counter = Counter()
t_counter = Counter()
c_counter = Counter()
for i , l in enumerate(list82):
    tc_counter.update([l])
    v = l.split("\t")
    t_counter.update([v[0]])
    c_counter.update([v[1]])
    if i > 999999999:
        break
    

In [ ]:
v[0]

In [ ]:
c_counter.most_common()

In [ ]:
with open(fname_tc_counter83 , "wb") as wf:
    pickle.dump(tc_counter , wf)

In [ ]:
with open(fname_t_counter83 , "wb") as wf:
    pickle.dump(t_counter , wf)

In [ ]:
with open(fname_c_counter83 , "wb") as wf:
    pickle.dump(c_counter , wf)

In [ ]:
%%bash
cd nlp100data

ls -l   *83.pkl

In [ ]:
%%bash
git commit -a -m "doing 84"

##### counterの使い方

###### unique elementの数

In [ ]:

len(t_counter)

In [ ]:
len(tc_counter)

In [ ]:
len(c_counter)


###### key , countの抽出

In [ ]:
tmp = t_counter.items()

In [ ]:
list(tmp)

In [ ]:
list(t_counter.keys())[0:10]

In [ ]:
list(t_counter.values())[0:10]

###### count合計

In [ ]:
sum(t_counter.values())

In [ ]:
sum(tc_counter.values())

##### sandbox

In [ ]:
import ngram

In [ ]:
t_counter['Personal']

#### 単語文脈行列の作成
83の出力を利用し，単語文脈行列$X$を作成せよ．ただし，行列$X$の各要素$X_{tc}$は次のように定義する．

- $f(t,c)≥10$ならば，$X_{tc}= {\rm PPMI}(t,c) == \max\{\log \frac{N×f(t,c)}{f(t,∗)×f(∗,c)},0\}$
- $f(t,c)<10$ならば，$X_{tc}=0$


ここで，${\rm PPMI}(t,c)$はPositive Pointwise Mutual Information（正の相互情報量）と呼ばれる統計量である．

なお，行列$X$の行数・列数は数百万オーダとなり，行列のすべての要素を主記憶上に載せることは無理なので注意すること．幸い，行列XXのほとんどの要素は0になるので，非0の要素だけを書き出せばよい．

In [13]:
fname_matrix84 = "nlp100data/matrix84"

In [14]:
fname_dict_t = "nlp100data/dict_t84.pkl"
fname_dict_c = "nlp100data/dict_c84.pkl"

In [17]:
import math

In [15]:
from scipy import io

In [16]:
from scipy.sparse import lil_matrix

In [ ]:
with open(fname_tc_counter83 , "rb") as rf:
    tc_counter = pickle.load(rf)

In [ ]:
with open(fname_t_counter83 , "rb") as rf:
    t_counter = pickle.load(rf)

In [ ]:
with open(fname_c_counter83 , "rb") as rf:
    c_counter = pickle.load(rf)

In [ ]:
N = sum(tc_counter.values())

In [ ]:
t_list = list(t_counter.keys())

In [ ]:
c_list = list(c_counter.keys())

In [ ]:
size_t = len(t_list)

In [ ]:
size_c = len(c_list)

In [ ]:
matrix84 = lil_matrix((size_t , size_c))

In [ ]:
from collections import OrderedDict

In [ ]:
dict_t = OrderedDict()
for i , t in enumerate(t_counter.keys()):
    dict_t[t] = i

In [ ]:
dict_c = OrderedDict()
for i , c in enumerate(c_counter.keys()):
    dict_c[c] = i

In [ ]:
for k , f_tc in tc_counter.items():
    if f_tc >= 10:
        #print(".")
        tokens = k.split("\t")
        t = tokens[0]
        c = tokens[1]
        i = dict_t[t]
        j = dict_c[c]
        ppmi = max(math.log(N * f_tc) / (t_counter[t] * c_counter[c]), 0)
        matrix84[i , j] = ppmi

In [ ]:
matrix84.shape

In [ ]:
io.savemat(fname_matrix84 , {'matrix84' : matrix84})

In [ ]:
with open(fname_dict_t , "wb") as wf:
    pickle.dump(dict_t , wf)

In [ ]:
with open(fname_dict_c , "wb") as wf:
    pickle.dump(dict_c , wf)

##### sparse matrix types

[解説](https://datascience.stackexchange.com/questions/31352/understanding-scipy-sparse-matrix-types)

```
lilとかは行列をセットするときに使い、CSRなどに変換してから各種演算を行え
```
とある

[解説２](http://d.hatena.ne.jp/billest/20090906/1252269157)
saveにはio.savematを使う旨書いてある。

##### sandbox

In [ ]:
dict_t = dict()
for i , t in enumerate(t_counter.keys()):
    dict_t[t] = i

In [ ]:
dict_c = dict()
for i , c in enumerate(c_counter.keys()):
    dict_c[c] = i

In [ ]:
for k , f_tc in tc_counter.items():
    if f_tc >= 10:
        tokens = k.split("\t")
        t = tokens[0]
        c = tokens[1]
        i = t_list.index(t)
        j = c_list.index(c)
        ppmi = max(math.log(N * f_tc) / (t_counter[t] * c_counter[c]), 0)
        matrix84[i , j] = ppmi

In [ ]:
%%bash
git commit -a -m "84 done"

#### 主成分分析による次元圧縮

84で得られた単語文脈行列に対して，主成分分析を適用し，単語の意味ベクトルを300次元に圧縮せよ．

[いろいろ手法がある](http://jakevdp.github.io/blog/2012/12/19/sparse-svds-in-python/)

In [18]:
matrix84 = io.loadmat(fname_matrix84)['matrix84']

In [19]:
matrix84.shape

(388634, 10636393)

##### scipy.linalg.svdによる方法

そもそも対応してない

In [26]:
from scipy.linalg import svd

In [27]:
u , s ,vt = svd(matrix84, 50)

ValueError: Sparse matrices are not supported by this function. Perhaps one of the scipy.sparse.linalg functions would work instead.

##### scipy.sparse.linalgによる方法

https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.linalg.svds.html

良さそうにみえるがメモリエラーになりがち

In [20]:
from scipy.sparse.linalg import svds, eigs

In [21]:
type(matrix84)

scipy.sparse.csc.csc_matrix

In [22]:
matrix84.asfptype()

<388634x10636393 sparse matrix of type '<class 'numpy.float64'>'
	with 7356 stored elements in Compressed Sparse Column format>

In [24]:
u,s,vt = svds(matrix84,50)

MemoryError: 

In [ ]:
u.shape

In [ ]:
s.shape

In [ ]:
vt.shape

##### よく言及されている方法

アウトプットは列数が圧縮された行列。？？

In [28]:
from sklearn.decomposition import TruncatedSVD

In [29]:
clf = TruncatedSVD(2)

In [30]:
matrix85 = clf.fit_transform(matrix84)


In [31]:
type(matrix85)

numpy.ndarray

In [32]:
matrix85.shape

(388634, 2)